# Flip Opportunity Scoring Analysis

This notebook applies the flip opportunity scoring model to identify high-potential markets.

## Scoring Components:
1. **Appreciation Score** - ZIP-level price growth (12-month lookback)
2. **Velocity Score** - Days to pending (faster = better for flippers)
3. **Distress Score** - Price cut percentage (higher = motivated sellers)
4. **Pricing Power Score** - Sale-to-list ratio (lower = buyer advantage)
5. **Value Gap Score** - Difference between all-homes and bottom-tier (renovation potential)

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from src.data_loader import load_all_datasets
from src.scoring_engine import (
    flip_opportunity_score,
    get_score_breakdown,
    filter_opportunities,
    summarize_by_geography,
    FAST_FLIP, VALUE_ADD_FLIP, BALANCED
)

pd.set_option('display.max_columns', 25)
pd.set_option('display.width', 200)
pd.set_option('display.float_format', '{:.2f}'.format)

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print('Loading datasets...')
datasets = load_all_datasets()
print('Done!')

## 1. Generate Opportunity Scores

In [ ]:
# Score all ZIPs using balanced strategy
print('Calculating flip opportunity scores...')
print(f'Strategy: {BALANCED.name}')
print(f'Weights: Appreciation={BALANCED.appreciation_weight}, Velocity={BALANCED.velocity_weight}, '
      f'Distress={BALANCED.distress_weight}, Pricing={BALANCED.pricing_power_weight}, '
      f'ValueGap={BALANCED.value_gap_weight}')
print()

scores = flip_opportunity_score(
    datasets=datasets,
    strategy=BALANCED,
    min_home_value=50000,
    max_home_value=500000
)

print(f'Total ZIPs scored: {len(scores):,}')
print(f'Score range: {scores["composite_score"].min():.1f} - {scores["composite_score"].max():.1f}')
print(f'Mean score: {scores["composite_score"].mean():.1f}')
print(f'Median score: {scores["composite_score"].median():.1f}')

## 2. Top 50 Opportunity ZIPs

In [ ]:
print('='*100)
print('TOP 50 FLIP OPPORTUNITIES')
print('='*100)

top_50 = scores.head(50).copy()
top_50['rank'] = range(1, 51)

display_cols = ['rank', 'region_name', 'city', 'state', 'metro', 
                'current_value', 'composite_score',
                'appreciation_score', 'velocity_score', 'distress_score']
print(top_50[display_cols].to_string(index=False))

In [ ]:
# Detailed breakdown for top 5
print('\n' + '='*80)
print('DETAILED BREAKDOWN - TOP 5 OPPORTUNITIES')
print('='*80)

for i, row in scores.head(5).iterrows():
    print(f"\n#{scores.index.get_loc(i)+1}: ZIP {row['region_name']} - {row['city']}, {row['state']}")
    print(f"   Metro: {row['metro']}")
    print(f"   Current Value: ${row['current_value']:,.0f}")
    print(f"   COMPOSITE SCORE: {row['composite_score']:.1f}/100")
    print(f"   ---")
    print(f"   Appreciation: {row['appreciation_score']:.1f}/100 ({row['appreciation_pct']:.1f}% 12mo growth)")
    print(f"   Velocity: {row['velocity_score']:.1f}/100 ({row.get('days_to_pending', 'N/A')} days to pending)")
    print(f"   Distress: {row['distress_score']:.1f}/100 ({row.get('price_cut_pct', 'N/A'):.1f}% price cuts)")
    print(f"   Pricing Power: {row['pricing_power_score']:.1f}/100 ({row.get('sale_to_list', 'N/A'):.3f} sale/list)")
    print(f"   Value Gap: {row['value_gap_score']:.1f}/100 ({row.get('value_gap_pct', 'N/A'):.1f}% gap)")

## 3. Score Distribution Analysis

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Composite score distribution
ax = axes[0, 0]
ax.hist(scores['composite_score'], bins=50, edgecolor='black', alpha=0.7)
ax.axvline(scores['composite_score'].mean(), color='red', linestyle='--', label=f'Mean: {scores["composite_score"].mean():.1f}')
ax.axvline(scores['composite_score'].median(), color='green', linestyle='--', label=f'Median: {scores["composite_score"].median():.1f}')
ax.set_xlabel('Composite Score')
ax.set_ylabel('Count')
ax.set_title('Composite Score Distribution')
ax.legend()

# Component score distributions
components = [
    ('appreciation_score', 'Appreciation Score'),
    ('velocity_score', 'Velocity Score'),
    ('distress_score', 'Distress Score'),
    ('pricing_power_score', 'Pricing Power Score'),
    ('value_gap_score', 'Value Gap Score')
]

for idx, (col, title) in enumerate(components):
    ax = axes[(idx+1)//3, (idx+1)%3]
    data = scores[col].dropna()
    ax.hist(data, bins=40, edgecolor='black', alpha=0.7)
    ax.axvline(data.mean(), color='red', linestyle='--', alpha=0.7)
    ax.set_xlabel('Score')
    ax.set_ylabel('Count')
    ax.set_title(f'{title} (mean: {data.mean():.1f})')

plt.tight_layout()
plt.savefig('../data/processed/score_distributions.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: data/processed/score_distributions.png')

## 4. Geographic Analysis

In [ ]:
# Summary by state
print('='*80)
print('TOP 20 STATES BY AVERAGE OPPORTUNITY SCORE')
print('='*80)

state_summary = summarize_by_geography(scores, level='state')
print(state_summary.head(20).to_string())

In [ ]:
# Visualize top states
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Top 15 states by average score
ax = axes[0]
top_states = state_summary.head(15)
ax.barh(range(len(top_states)), top_states['avg_score'], color='steelblue')
ax.set_yticks(range(len(top_states)))
ax.set_yticklabels(top_states.index)
ax.invert_yaxis()
ax.set_xlabel('Average Opportunity Score')
ax.set_title('Top 15 States by Avg Score')

# Top 15 states by number of opportunities
ax = axes[1]
top_count = state_summary.sort_values('num_opportunities', ascending=False).head(15)
ax.barh(range(len(top_count)), top_count['num_opportunities'], color='coral')
ax.set_yticks(range(len(top_count)))
ax.set_yticklabels(top_count.index)
ax.invert_yaxis()
ax.set_xlabel('Number of Opportunities')
ax.set_title('Top 15 States by # of Opportunities')

plt.tight_layout()
plt.savefig('../data/processed/state_analysis.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: data/processed/state_analysis.png')

In [ ]:
# Summary by metro
print('='*80)
print('TOP 25 METROS BY AVERAGE OPPORTUNITY SCORE')
print('='*80)

metro_summary = summarize_by_geography(scores, level='metro')
# Filter to metros with at least 5 ZIPs
metro_summary_filtered = metro_summary[metro_summary['num_opportunities'] >= 5]
print(metro_summary_filtered.head(25).to_string())

In [ ]:
# Top metros visualization
fig, ax = plt.subplots(figsize=(12, 8))

top_metros = metro_summary_filtered.head(20)
colors = plt.cm.RdYlGn(top_metros['avg_score'] / 100)

bars = ax.barh(range(len(top_metros)), top_metros['avg_score'], color=colors)
ax.set_yticks(range(len(top_metros)))
ax.set_yticklabels([m[:40] + '...' if len(m) > 40 else m for m in top_metros.index])
ax.invert_yaxis()
ax.set_xlabel('Average Opportunity Score')
ax.set_title('Top 20 Metros by Average Opportunity Score (min 5 ZIPs)')

# Add count labels
for i, (score, count) in enumerate(zip(top_metros['avg_score'], top_metros['num_opportunities'])):
    ax.text(score + 0.5, i, f'n={count}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig('../data/processed/metro_analysis.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: data/processed/metro_analysis.png')

## 5. Component Correlation Analysis

In [ ]:
# Correlation between score components
score_cols = ['composite_score', 'appreciation_score', 'velocity_score', 
              'distress_score', 'pricing_power_score', 'value_gap_score']

corr_matrix = scores[score_cols].corr()

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='RdYlGn', center=0,
            fmt='.2f', square=True, ax=ax)
ax.set_title('Score Component Correlations')
plt.tight_layout()
plt.savefig('../data/processed/score_correlations.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: data/processed/score_correlations.png')

## 6. Strategy Comparison

In [ ]:
# Compare different strategies
print('='*80)
print('STRATEGY COMPARISON')
print('='*80)

strategies = [FAST_FLIP, VALUE_ADD_FLIP, BALANCED]
strategy_results = {}

for strategy in strategies:
    strat_scores = flip_opportunity_score(
        datasets=datasets,
        strategy=strategy,
        min_home_value=50000,
        max_home_value=500000
    )
    strategy_results[strategy.name] = strat_scores
    
    print(f"\n{strategy.name}:")
    print(f"  Weights: App={strategy.appreciation_weight}, Vel={strategy.velocity_weight}, "
          f"Dist={strategy.distress_weight}, Price={strategy.pricing_power_weight}, "
          f"Gap={strategy.value_gap_weight}")
    print(f"  Top 5 ZIPs:")
    for _, row in strat_scores.head(5).iterrows():
        print(f"    {row['region_name']} ({row['city']}, {row['state']}): {row['composite_score']:.1f}")

In [ ]:
# Compare top 10 ZIPs across strategies
print('\n' + '='*80)
print('TOP 10 COMPARISON ACROSS STRATEGIES')
print('='*80)

comparison_df = pd.DataFrame()

for name, strat_scores in strategy_results.items():
    top10 = strat_scores.head(10)[['region_name', 'city', 'state', 'composite_score']].copy()
    top10['strategy'] = name
    top10['rank'] = range(1, 11)
    comparison_df = pd.concat([comparison_df, top10])

# Pivot to show side by side
comparison_pivot = comparison_df.pivot(index='rank', columns='strategy', 
                                        values=['region_name', 'composite_score'])
print(comparison_pivot.to_string())

## 7. Filtered Opportunity Analysis

In [ ]:
# High-score opportunities (60+) with specific characteristics
print('='*80)
print('FILTERED OPPORTUNITIES: Score >= 65, Fast Markets (< 45 days)')
print('='*80)

fast_market_opps = filter_opportunities(
    scores,
    min_score=65,
    max_days_to_pending=45
)

print(f"Found {len(fast_market_opps):,} opportunities")
print()
print(fast_market_opps.head(20)[['region_name', 'city', 'state', 'metro', 
                                  'current_value', 'composite_score', 
                                  'days_to_pending']].to_string(index=False))

In [ ]:
# Distressed market opportunities
print('='*80)
print('FILTERED OPPORTUNITIES: Score >= 60, High Distress (> 25% price cuts)')
print('='*80)

distressed_opps = filter_opportunities(
    scores,
    min_score=60,
    min_price_cuts=25
)

print(f"Found {len(distressed_opps):,} opportunities")
print()
print(distressed_opps.head(20)[['region_name', 'city', 'state', 'metro', 
                                 'current_value', 'composite_score', 
                                 'price_cut_pct']].to_string(index=False))

In [ ]:
# High appreciation potential
print('='*80)
print('FILTERED OPPORTUNITIES: Score >= 55, High Appreciation (> 5% YoY)')
print('='*80)

appreciation_opps = filter_opportunities(
    scores,
    min_score=55,
    min_appreciation=5
)

print(f"Found {len(appreciation_opps):,} opportunities")
print()
print(appreciation_opps.head(20)[['region_name', 'city', 'state', 'metro', 
                                   'current_value', 'composite_score', 
                                   'appreciation_pct']].to_string(index=False))

## 8. Characteristics of High-Scoring Areas

In [ ]:
# Compare characteristics: Top 20% vs Bottom 20%
print('='*80)
print('CHARACTERISTICS COMPARISON: TOP 20% vs BOTTOM 20%')
print('='*80)

top_20_pct = scores.head(int(len(scores) * 0.2))
bottom_20_pct = scores.tail(int(len(scores) * 0.2))

metrics = ['current_value', 'appreciation_pct', 'days_to_pending', 
           'price_cut_pct', 'sale_to_list', 'value_gap_pct']

comparison = pd.DataFrame({
    'Metric': metrics,
    'Top 20% Mean': [top_20_pct[m].mean() for m in metrics],
    'Bottom 20% Mean': [bottom_20_pct[m].mean() for m in metrics],
    'Difference': [top_20_pct[m].mean() - bottom_20_pct[m].mean() for m in metrics]
})
print(comparison.to_string(index=False))

In [ ]:
# Visualize key differentiators
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

metrics_to_plot = [
    ('current_value', 'Home Value ($)', False),
    ('appreciation_pct', 'Appreciation (%)', False),
    ('days_to_pending', 'Days to Pending', False),
    ('price_cut_pct', 'Price Cut (%)', False),
    ('sale_to_list', 'Sale/List Ratio', False),
    ('composite_score', 'Composite Score', True)
]

for idx, (col, title, is_score) in enumerate(metrics_to_plot):
    ax = axes[idx//3, idx%3]
    
    # Create bins based on composite score quintiles
    scores['score_quintile'] = pd.qcut(scores['composite_score'], 5, labels=['Q1 (Low)', 'Q2', 'Q3', 'Q4', 'Q5 (High)'])
    
    quintile_means = scores.groupby('score_quintile')[col].mean()
    
    colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, 5))
    ax.bar(range(5), quintile_means.values, color=colors)
    ax.set_xticks(range(5))
    ax.set_xticklabels(['Q1\n(Low)', 'Q2', 'Q3', 'Q4', 'Q5\n(High)'])
    ax.set_ylabel(title)
    ax.set_title(f'{title} by Score Quintile')

plt.tight_layout()
plt.savefig('../data/processed/score_quintile_analysis.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: data/processed/score_quintile_analysis.png')

## 9. Export Top Opportunities

In [ ]:
# Create processed data directory if it doesn't exist
output_dir = Path('../data/processed')
output_dir.mkdir(parents=True, exist_ok=True)

# Export top 500 opportunities
top_opportunities = scores.head(500).copy()
top_opportunities['rank'] = range(1, 501)

# Reorder columns for export
export_cols = [
    'rank', 'region_name', 'city', 'state', 'metro', 'county_name',
    'current_value', 'composite_score',
    'appreciation_score', 'velocity_score', 'distress_score',
    'pricing_power_score', 'value_gap_score',
    'appreciation_pct', 'days_to_pending', 'price_cut_pct', 
    'sale_to_list', 'value_gap_pct', 'strategy'
]

export_cols = [c for c in export_cols if c in top_opportunities.columns]
top_opportunities = top_opportunities[export_cols]

# Save to CSV
output_path = output_dir / 'top_opportunities.csv'
top_opportunities.to_csv(output_path, index=False)
print(f'Exported top 500 opportunities to: {output_path}')
print(f'\nExported columns: {list(top_opportunities.columns)}')

In [ ]:
# Also export by strategy
for name, strat_scores in strategy_results.items():
    filename = f'top_opportunities_{name.lower().replace(" ", "_").replace("-", "_")}.csv'
    filepath = output_dir / filename
    
    export_df = strat_scores.head(500).copy()
    export_df['rank'] = range(1, 501)
    export_df = export_df[[c for c in export_cols if c in export_df.columns]]
    
    export_df.to_csv(filepath, index=False)
    print(f'Exported {name} top 500 to: {filepath}')

## 10. Summary & Key Findings

In [ ]:
print('='*80)
print('FLIP OPPORTUNITY ANALYSIS SUMMARY')
print('='*80)

print(f'''
ANALYSIS SCOPE:
- Total ZIPs analyzed: {len(scores):,}
- Value range: $50,000 - $500,000
- Time period: Latest available data (through Nov 2025)

SCORE DISTRIBUTION:
- Score range: {scores['composite_score'].min():.1f} - {scores['composite_score'].max():.1f}
- Mean score: {scores['composite_score'].mean():.1f}
- Median score: {scores['composite_score'].median():.1f}
- High-opportunity ZIPs (score >= 65): {len(scores[scores['composite_score'] >= 65]):,}
- Very high-opportunity ZIPs (score >= 75): {len(scores[scores['composite_score'] >= 75]):,}

TOP PERFORMING STATES:
''')
for state, row in state_summary.head(5).iterrows():
    print(f"  {state}: {row['avg_score']:.1f} avg score, {int(row['num_opportunities'])} ZIPs")

print(f'''
TOP PERFORMING METROS:
''')
for metro, row in metro_summary_filtered.head(5).iterrows():
    print(f"  {metro[:50]}: {row['avg_score']:.1f} avg score, {int(row['num_opportunities'])} ZIPs")

print(f'''
KEY CHARACTERISTICS OF HIGH-SCORING AREAS:
''')
top_20 = scores.head(int(len(scores) * 0.2))
print(f"  - Avg home value: ${top_20['current_value'].mean():,.0f}")
print(f"  - Avg 12-month appreciation: {top_20['appreciation_pct'].mean():.1f}%")
print(f"  - Avg days to pending: {top_20['days_to_pending'].mean():.0f}")
print(f"  - Avg price cuts: {top_20['price_cut_pct'].mean():.1f}%")
print(f"  - Avg sale/list ratio: {top_20['sale_to_list'].mean():.3f}")

print(f'''
EXPORTED FILES:
- data/processed/top_opportunities.csv (Top 500 balanced strategy)
- data/processed/top_opportunities_fast_flip.csv
- data/processed/top_opportunities_value_add_flip.csv
- data/processed/top_opportunities_balanced.csv
- data/processed/score_distributions.png
- data/processed/state_analysis.png
- data/processed/metro_analysis.png
- data/processed/score_correlations.png
- data/processed/score_quintile_analysis.png
''')